## Import Packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Ideas for Lane Detection Pipeline

In [ ]:
import pandas as pd
import math
import os

In [4]:
import pickle

# Training

In [8]:
from vehicle_detection import VehicleDetectionPipeline
from pipeline import ImageProcessor

In [ ]:
path = './data/full/*/*/*.png'

p = VehicleDetectionPipeline()

p.preprocess_data(path=path)

p.train()

pickle.dump(p.classifier, open('classifier-all-ycrcb.p','wb'))
pickle.dump(p.scaler, open('scaler-all-ycrcb.p','wb'))

classifier = p.classifier
scaler = p.scaler

In [6]:
classifier = pickle.load(open('classifier-all-ycrcb.p','rb'))
scaler = pickle.load(open('scaler-all-ycrcb.p','rb'))

# Video Test

In [10]:
import imageio
# imageio.plugins.ffmpeg.download()
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from functools import partial

def gen():
    for i in range(10,10000):
        yield i
        
g = gen()

def find_cars(image):
    ip = ImageProcessor(image)
    ip.run_find_cars(classifier, scaler)
    return ip.result_image

def extract_frames(image):
    result = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imwrite('./test_frames/frame_{}.jpg'.format(next(g)),result)
    return result


clip1 = VideoFileClip("project_video.mp4")
output_dir = "./output"
file_name = "project_video.mp4"

output_path = os.path.join(output_dir, file_name)
video_output_path = os.path.realpath(output_path)
output_clip = clip1.fl_image(find_cars).subclip(27,32)
# output_clip = clip1.fl_image(find_cars)

%time output_clip.write_videofile(video_output_path, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(output_path))

[MoviePy] >>>> Building video /Users/amogh/workspace/jazz/code/courses/sdc_project5/output/project_video.mp4
[MoviePy] Writing video /Users/amogh/workspace/jazz/code/courses/sdc_project5/output/project_video.mp4


 99%|█████████▉| 125/126 [04:34<00:02,  2.21s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /Users/amogh/workspace/jazz/code/courses/sdc_project5/output/project_video.mp4 

CPU times: user 4min 26s, sys: 7.31 s, total: 4min 33s
Wall time: 4min 34s


# All Test Images

In [ ]:
from pipeline import ImageProcessor

import glob
image_list = glob.glob('./test_images/*.jpg')

classifier = pickle.load(open('classifier-all-hog.p','rb'))
scaler = pickle.load(open('scaler-all-hog.p','rb'))

image_list

for i in range(0,len(image_list)):
    image_path = image_list[i]
    ip = ImageProcessor('{}'.format(image_path))
    ip.run_find_cars(classifier, scaler)
    break